In [14]:
import pandas as pd
import numpy as np

In [9]:
df = pd.read_csv("seatsurveymodel.csv")

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6,7,8,25,26,60,72,84,96,108,120,132,144,156,168,180,192,204,216,228,240,252,264,276,283,288,289,290) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
df.columns

Index(['participant.id_in_session', 'participant.code', 'participant.label',
       'participant._is_bot', 'participant._index_in_pages',
       'participant._max_page_index', 'participant._current_app_name',
       'participant._current_page_name', 'participant.time_started',
       'participant.visited',
       ...
       'survey.1.player.gender', 'survey.1.player.feet',
       'survey.1.player.inches', 'survey.1.player.rt',
       'survey.1.player.bizlei', 'survey.1.player.seattype',
       'survey.1.player.section', 'survey.1.player.comment',
       'survey.1.group.id_in_subsession', 'survey.1.subsession.round_number'],
      dtype='object', length=293)

In [12]:
df.drop(['participant.code','participant.label','participant._is_bot',
        'participant._current_app_name','participant._current_page_name','participant.visited',
        'participant.mturk_worker_id','participant.mturk_assignment_id','participant.payoff',
        'session.code','session.label','session.mturk_HITId','session.mturk_HITGroupId','session.comment',
        'session.is_demo'], inplace = True, axis = 1
       )

In [13]:
df.columns

Index(['participant.id_in_session', 'participant._index_in_pages',
       'participant._max_page_index', 'participant.time_started',
       'session.config.real_world_currency_per_point',
       'session.config.participation_fee', 'randseq.1.player.id_in_group',
       'randseq.1.player.role', 'randseq.1.player.payoff',
       'randseq.1.player.page_sequence',
       ...
       'survey.1.player.gender', 'survey.1.player.feet',
       'survey.1.player.inches', 'survey.1.player.rt',
       'survey.1.player.bizlei', 'survey.1.player.seattype',
       'survey.1.player.section', 'survey.1.player.comment',
       'survey.1.group.id_in_subsession', 'survey.1.subsession.round_number'],
      dtype='object', length=278)